In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('../')

import skeletor as sk

import tqdm

import networkx as nx

from anytree import Node, RenderTree

In [ ]:
data = 'Data/cube.npy'
dsFactor = 1
neighborDistance = 25
np.random.seed(10)

#%matplotlib qt
with open(data, 'rb') as f:
    rawData = np.load(f)
    
order = np.arange(rawData.shape[0])
np.random.shuffle(order)
points = rawData[order][::dsFactor,:]

#plt.scatter(scatterPoints[:,0], scatterPoints[:,2], s=.2)
#plt.show()

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter(points[:,0], points[:,1], points[:,2], s=1)
plt.show()

In [ ]:
octree = sk.Octree(points, 5000, 1/128, neighborMethod='discrete')
skelPoints, skelAdjMat = octree.skeletonize(threshold=-.7, closed=True, mergeThreshold=1, secondRoundNeighborDetection=True, allowMergePoints=True)

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(8,8))

fourCycles = sk.identifyCycles(skelAdjMat, 4)

ax = fig.add_subplot(projection='3d')

ax.scatter(skelPoints[:,0], skelPoints[:,1], skelPoints[:,2], s=2, alpha=1)
for i in range(len(skelAdjMat)):
    edgeIndices = np.where(skelAdjMat[i] > 0)[0]
    for j in range(len(edgeIndices)):
        ax.plot(*list(zip(skelPoints[i], skelPoints[edgeIndices[j]])))
        
for i in range(len(fourCycles)):
    for j in range(len(fourCycles[i])):
        ax.plot(*list(zip(skelPoints[fourCycles[i][j]], skelPoints[fourCycles[i][(j+1)%4]])), c='tab:red')

fig.tight_layout()
plt.savefig('four_cycles.png', bbox_inches='tight')
plt.show()

In [ ]:
plt.hist(sk.computeCyclePathLength(skelPoints, fourCycles))
plt.xlabel('Cycle length')
plt.ylabel('Frequency')
plt.show()

In [ ]:
def collapseCycles(points, adjMat, N, distanceThreshold=.2):
    """
    """
    
    currentPoints = points
    currentAdjMat = adjMat
    
    newPoints = []
    
    # Compute cycles
    cycleArr = sk.identifyCycles(currentAdjMat, N)

    # Compute path lengths
    pathLengthArr = sk.computeCyclePathLength(currentPoints, cycleArr)

    # Find which are less than the distance threshold
    removableCycles = np.where(pathLengthArr < distanceThreshold)[0]

    # Find the indices of all of the points NOT involved in
    # these cycles
    allCyclePoints = np.unique(cycleArr[removableCycles])
    unchangedPointIndices = [i for i in range(len(currentPoints)) if i not in allCyclePoints]

    newPoints += list(currentPoints[unchangedPointIndices])

    # Now take the center of each cycle as a new point to replace the
    # original points involved in that cycle. Order doesn't matter
    # since adjacent cycles will lead to two new points, even if
    # some number of vertices are common between the two
    newPoints += [np.mean(currentPoints[cyc], axis=0) for cyc in cycleArr[removableCycles]]

    # Add all unchanged points
    newToOldConversion = {j : [unchangedPointIndices[j]] for j in range(len(unchangedPointIndices))}
    # Add all merged points
    newToOldConversion.update({len(unchangedPointIndices) + j : list(cycleArr[removableCycles[j]]) for j in range(len(removableCycles))})

    # Now make the complementary conversion dictionary
    # The where part is picking out where the index shows up
    # in the values of the other dictionary, eg. the value for the key 0
    # (the new points corresponding to the old point 0) will be a list of
    # keys from newToOldConversion for which 0 shows up as a value.
    oldToNewConversion = {j : list(np.where([j in newToOldConversion[k] for k in newToOldConversion])[0]) for j in range(len(currentPoints))}

    # print(newToOldConversion)
    # print()
    # print(oldToNewConversion)
    #break

    # Update the adjacency matrix
    oldEdges = np.where(currentAdjMat > 0)

    mergedPointAncestors = cycleArr[removableCycles]
    newAdjMat = np.zeros((len(newPoints), len(newPoints)))

    # Unchanged points can just be copied over from the previous adjacency matrix
    newAdjMat[:len(unchangedPointIndices),:len(unchangedPointIndices)] = adjMat[unchangedPointIndices][:,unchangedPointIndices]

    for i in range(len(mergedPointAncestors)):
        newNeighbors = np.array([oldToNewConversion[j] for j in mergedPointAncestors[i]])
        # Flatten (for some reason, numpy's flatten isn't working here)
        newNeighbors = np.unique([n for neigh in newNeighbors for n in neigh])
        oldNeighbors = [newToOldConversion[n] for n in newNeighbors]
        # Flatten
        oldNeighbors = np.unique([n for neigh in oldNeighbors for n in neigh])

        oldConnections = np.where(np.sum([adjMat[n] for n in oldNeighbors], axis=0) > 0)[0]
        newConnections = [oldToNewConversion[o] for o in oldConnections]
        # Flatten
        newConnections = [c for newConn in newConnections for c in newConn]
        adjMatRow = [i in newConnections for i in range(len(newPoints))]

        #print(np.shape(adjMatRow))
        newAdjMat[i+len(unchangedPointIndices)] += adjMatRow

    currentPoints = np.array(newPoints)
    currentAdjMat = newAdjMat

    return currentPoints, currentAdjMat

In [ ]:
def collapseCycles(points, adjMat, N, distanceThreshold=.2):
    """
    """
    
    currentPoints = points.copy()
    currentAdjMat = adjMat.copy()
    
    newPoints = []
    
    # Compute cycles
    cycleArr = sk.identifyCycles(currentAdjMat, N)

    # Compute path lengths
    pathLengthArr = sk.computeCyclePathLength(currentPoints, cycleArr)
    
    # Order from smallest to largest, so we remove smallest cycles first
    order = np.argsort(pathLengthArr)
    pathLengthArr = pathLengthArr[order]
    cycleArr = cycleArr[order]
    
    # Find which are less than the distance threshold
    removableCycles = np.where(pathLengthArr < distanceThreshold)[0]

    # Now iteratively remove cycles, starting from the smallest ones
    for i in range(len(removableCycles)):
        cyclePointIndices = cycleArr[removableCycles[i]]
        midpoint = np.mean(currentPoints[cyclePointIndices], axis=0)
        
        adjMatRow = (np.sum(adjMat[cyclePointIndices], axis=0) > 0).astype(np.int64)
        #print(adjMatRow)
        # Assign new midpoint to where the old points were, and update the adjacency matrix
        currentPoints[cyclePointIndices] = midpoint
        for cpi in cyclePointIndices:
            currentAdjMat[cpi] = adjMatRow
            currentAdjMat[:,cpi] = adjMatRow

        
    # Finally, take only the unique points
    _, finalPointIndices = np.unique(currentPoints, axis=0, return_index=True)
    #print(finalPointIndices)
    
    finalPoints = currentPoints[finalPointIndices]
    finalAdjMat = currentAdjMat[finalPointIndices][:,finalPointIndices]
    
    return finalPoints, finalAdjMat

In [ ]:
newPoints, newAdjMat = collapseCycles(skelPoints, skelAdjMat, 4)
#newPoints, newAdjMat = collapseCycles(newPoints, newAdjMat, 3)

fig = plt.figure(figsize=(8,8))

ax = fig.add_subplot(projection='3d')

ax.scatter(newPoints[:,0], newPoints[:,1], newPoints[:,2], s=2, alpha=1)
for i in range(len(newAdjMat)):
    edgeIndices = np.where(newAdjMat[i] > 0)[0]
    for j in range(len(edgeIndices)):
        ax.plot(*list(zip(newPoints[i], newPoints[edgeIndices[j]])))
        
#for i in range(len(fourCycles)):
#    for j in range(len(fourCycles[i])):
#        ax.plot(*list(zip(skelPoints[fourCycles[i][j]], skelPoints[fourCycles[i][(j+1)%4]])), c='tab:red')

plt.show()

In [ ]:
testAdjMat = np.array([[0, 1, 0, 1, 1, 1],
                       [1, 0, 0, 0, 1, 0],
                       [0, 0, 0, 1, 0, 0],
                       [1, 0, 1, 0, 0, 0],
                       [1, 1, 0, 0, 0, 1],
                       [1, 0, 0, 0, 1, 0]])

sk.identifyCycles(testAdjMat, 4)